In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Solving environment: done

# All requested packages already installed.



ModuleNotFoundError: No module named 'folium'

In [3]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print("Libraries imported.")

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         713 KB

The following NEW packages will be INSTALLED:

    altair:  4.1.0-py_1 conda-forge
    branca:  0.4.1-py_0 conda-forge
    folium:  0.5.0-py_0 conda-forge
    vincent: 0.4.4-py_1 conda-forge


vincent-0.4.4        | 28 KB     | ##################################### | 1

<h3> Scrap data from Wikipedia page into a DataFrame</h3>

In [4]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_of_Chennai").text

In [5]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [6]:
# create a list to store neighborhood data
neighborhoodList = []

In [7]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [8]:
# create a new DataFrame from the list
c_df = pd.DataFrame({"Neighborhood": neighborhoodList})

c_df.head()

,Neighborhood
0,Alandur
1,Anna Nagar
2,"Ashok Nagar, Chennai"
3,Assisi Nagar
4,Ayanavaram


In [10]:
c_df.shape

(68, 1)

In [15]:
!pip install geocoder
import geocoder # to get coordinates


     |████████████████████████████████| 102kB 10.3MB/s ta 0:00:01


<h3>Get the geographical coordinates</h3>

In [16]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Chennai,India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [17]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in c_df["Neighborhood"].tolist() ]

In [20]:
coords

[[13.00013000000007, 80.20060000000007],
 [13.083590000000072, 80.21020000000004],
 [13.035390000000064, 80.21220000000005],
 [13.164570000000026, 80.23274000000004],
 [13.09883000000002, 80.23238000000003],
 [13.074621704974275, 80.24277657646144],
 [12.932770000000062, 80.14387000000005],
 [12.95234000000005, 80.14411000000007],
 [12.988610000000051, 80.15100000000007],
 [12.82725000000005, 80.22866000000005],
 [12.837900000000047, 80.05327000000005],
 [13.040920000000028, 80.13649000000004],
 [13.11035000000004, 80.21301000000005],
 [13.129720000000077, 80.18300000000005],
 [13.120580000000075, 80.06047000000007],
 [12.956150000000036, 80.17885000000007],
 [12.793410000000051, 80.22010000000006],
 [13.081980000000044, 80.24448000000007],
 [13.051520000000039, 80.22421000000008],
 [13.136630000000025, 80.24479000000008],
 [13.14141357139813, 80.19406933326228],
 [13.096050000000048, 80.05292000000009],
 [13.116800000000069, 80.27726000000007],
 [13.183260000000075, 80.24059000000005]

In [21]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [22]:
# merge the coordinates into the original dataframe
c_df['Latitude'] = df_coords['Latitude']
c_df['Longitude'] = df_coords['Longitude']

In [23]:
print(c_df.shape)
c_df

(68, 3)


,Neighborhood,Latitude,Longitude
0,Alandur,13.000130,80.200600
1,Anna Nagar,13.083590,80.210200
2,"Ashok Nagar, Chennai",13.035390,80.212200
3,Assisi Nagar,13.164570,80.232740
4,Ayanavaram,13.098830,80.232380
5,Chennai city,13.074622,80.242777
6,Chitlapakkam,12.932770,80.143870
7,Chromepet,12.952340,80.144110
8,Cowl Bazaar,12.988610,80.151000
9,Egattur (Kanchipuram District),12.827250,80.228660


In [24]:
# save the DataFrame as CSV file
c_df.to_csv("c_df.csv", index=False)

<h3>Create a map of Chennai with neighborhoods superimposed on top</h3>

In [26]:
# get the coordinates Chennai
address = 'Chennai, India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Chennai, India are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Chennai, India are 13.0801721, 80.2838331.


In [28]:
# create map of Chennai using latitude and longitude values
map_c = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(c_df['Latitude'], c_df['Longitude'], c_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_c)  
    
map_c

In [29]:
# save the map as HTML file
map_c.save('map_c.html')

<h3>Use the Foursquare API to explore the neighborhoods</h3>

In [30]:
# @hidden cell
# define Foursquare Credentials and Version
CLIENT_ID = 'HU1IF5IUM5M4QRJ1UJKGZKRNM24O3PDJ4G1DVXE2SUXL54TF' # your Foursquare ID
CLIENT_SECRET = 'NM2XHXM2SPBP2VKE0PQ4ZI442CEVVEDG4B1PVD5H1RNQJN5U' # your Foursquare Secret
VERSION = '20200518' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HU1IF5IUM5M4QRJ1UJKGZKRNM24O3PDJ4G1DVXE2SUXL54TF
CLIENT_SECRET:NM2XHXM2SPBP2VKE0PQ4ZI442CEVVEDG4B1PVD5H1RNQJN5U


<h4>Now, let's get the top 100 venues that are within a radius of 2000 meters.</h4>

In [31]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(c_df['Latitude'], c_df['Longitude'], c_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [32]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(1325, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Alandur,13.00013,80.2006,Sukkkubai Beef Biryani Shop,12.998769,80.201381,Indian Restaurant
1,Alandur,13.00013,80.2006,Pizza Republic,12.990987,80.198613,Pizza Place
2,Alandur,13.00013,80.2006,Moon & Six Pence - The Irish Bar,13.007848,80.208152,Bar
3,Alandur,13.00013,80.2006,Q Bar,13.016606,80.204853,Restaurant
4,Alandur,13.00013,80.2006,Hilton,13.016621,80.204787,Hotel


<h4>Let's check how many venues were returned for each neighorhood</h4>

In [33]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Alandur,50,50,50,50,50,50
Anna Nagar,100,100,100,100,100,100
"Ashok Nagar, Chennai",68,68,68,68,68,68
Assisi Nagar,3,3,3,3,3,3
Ayanavaram,24,24,24,24,24,24
Chennai city,100,100,100,100,100,100
Chitlapakkam,16,16,16,16,16,16
Chromepet,16,16,16,16,16,16
Cowl Bazaar,24,24,24,24,24,24


<h4>Let's find out how many unique categories can be curated from all the returned venues</h4>

In [34]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 157 uniques categories.


In [37]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:200]

array(['Indian Restaurant', 'Pizza Place', 'Bar', 'Restaurant', 'Hotel',
       'Church', 'Train Station', 'Asian Restaurant', 'Ice Cream Shop',
       'Donut Shop', 'South Indian Restaurant', "Men's Store",
       'Breakfast Spot', 'Bakery', 'Italian Restaurant',
       'Multicuisine Indian Restaurant', 'Pool Hall',
       'Japanese Restaurant', 'Supermarket', 'Bus Station', 'Café',
       'Juice Bar', 'Fast Food Restaurant', 'Department Store', 'Gym',
       'Dessert Shop', 'Cafeteria', 'Golf Course', 'Food Court',
       'Sandwich Place', 'Park', 'Chinese Restaurant', 'Snack Place',
       'Coffee Shop', 'Vegetarian / Vegan Restaurant', 'Shoe Store',
       'Indian Sweet Shop', 'American Restaurant', 'Burger Joint',
       'Middle Eastern Restaurant', 'Shopping Mall', 'Market',
       'BBQ Joint', 'New American Restaurant', 'Clothing Store',
       'Paper / Office Supplies Store', 'Jewelry Store', 'Multiplex',
       'Furniture / Home Store', 'Farmers Market', 'Bookstore', 'Bistro',

In [38]:
# check if the results contain "Shopping Mall"
"Neighborhood" in venues_df['VenueCategory'].unique()

False

<h3>Analyze Each Neighborhood</h3>

In [39]:
# one hot encoding
c_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
c_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [c_onehot.columns[-1]] + list(c_onehot.columns[:-1])
c_onehot = c_onehot[fixed_columns]

print(c_onehot.shape)
c_onehot.head()

(1325, 158)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Airport Food Court,Airport Lounge,Airport Terminal,American Restaurant,Andhra Restaurant,Antique Shop,Arcade,Asian Restaurant,Athletics & Sports,BBQ Joint,Badminton Court,Bakery,Bar,Beach,Big Box Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Buffet,Burger Joint,Bus Line,Bus Station,Cafeteria,Café,Campground,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Coffee Shop,College Cafeteria,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Daycare,Department Store,Dessert Shop,Diner,Donut Shop,Duty-free Shop,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden Center,Gas Station,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym Pool,Historic Site,Hookah Bar,Hospital,Hostel,Hotel,Hyderabadi Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Kerala Restaurant,Korean Restaurant,Lake,Light Rail Station,Lounge,Malay Restaurant,Market,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Motel,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,Music Venue,National Park,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Paper / Office Supplies Store,Park,Persian Restaurant,Pharmacy,Pizza Place,Platform,Playground,Pool Hall,Pub,Racetrack,Recreation Center,Resort,Rest Area,Restaurant,River,Road,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Snack Place,Soccer Stadium,South Indian Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Supermarket,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Train,Train Station,Travel & Transport,Vegetarian / Vegan Restaurant,Video Store,Whisky Bar,Women's Store,Zoo,Zoo Exhibit
0,Alandur,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Alandur,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Alandur,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Alandur,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Alandur,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


<h4>Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category>/h4>

In [40]:
c_grouped = c_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(c_grouped.shape)
c_grouped

(68, 158)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Airport Food Court,Airport Lounge,Airport Terminal,American Restaurant,Andhra Restaurant,Antique Shop,Arcade,Asian Restaurant,Athletics & Sports,BBQ Joint,Badminton Court,Bakery,Bar,Beach,Big Box Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Buffet,Burger Joint,Bus Line,Bus Station,Cafeteria,Café,Campground,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Coffee Shop,College Cafeteria,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Daycare,Department Store,Dessert Shop,Diner,Donut Shop,Duty-free Shop,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden Center,Gas Station,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym Pool,Historic Site,Hookah Bar,Hospital,Hostel,Hotel,Hyderabadi Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Kerala Restaurant,Korean Restaurant,Lake,Light Rail Station,Lounge,Malay Restaurant,Market,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Motel,Movie Theater,Moving Target,Multicuisine Indian Restaurant,Multiplex,Museum,Music Store,Music Venue,National Park,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Paper / Office Supplies Store,Park,Persian Restaurant,Pharmacy,Pizza Place,Platform,Playground,Pool Hall,Pub,Racetrack,Recreation Center,Resort,Rest Area,Restaurant,River,Road,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Snack Place,Soccer Stadium,South Indian Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Supermarket,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Train,Train Station,Travel & Transport,Vegetarian / Vegan Restaurant,Video Store,Whisky Bar,Women's Store,Zoo,Zoo Exhibit
0,Alandur,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.040000,0.00,0.000000,0.000000,0.020000,0.040000,0.000000,0.00,0.0,0.00,0.000,0.000000,0.00,0.040000,0.00,0.000000,0.000000,0.020000,0.020000,0.040000,0.000000,0.000000,0.00,0.02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.020000,0.020000,0.000000,0.020000,0.000000,0.00,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.020000,0.0,0.000000,0.0000,0.000000,0.00,0.000000,0.000000,0.020000,0.000000,0.000000,0.020000,0.000000,0.000000,0.00,0.000000,0.000000,0.120000,0.000000,0.000000,0.020000,0.180000,0.00,0.000000,0.000000,0.020000,0.00,0.02,0.00,0.020000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.02,0.000000,0.00,0.000000,0.000000,0.00,0.000,0.000000,0.00,0.000000,0.0,0.020000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.000000,0.040000,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000,0.040000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.040000,0.000000,0.00,0.000000,0.020000,0.00,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Anna Nagar,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.01,0.000000,0.000000,0.000000,0.010000,0.00,0.010000,0.000000,0.040000,0.000000,0.000000,0.00,0.0,0.01,0.000,0.010000,0.00,0.000000,0.00,0.010000,0.000000,0.010000,0.000000,0.040000,0.000000,0.030000,0.00,0.00,0.040000,0.040000,0.000000,0.010000,0.000000,0.000000,0.0,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.02,0.000000,0.010000,0.110000,0.000000,0.000000,0.010000,0.000000,0.010000,0.0,0.000000,0.0000,0.000000,0.01,0.000000,0.00

In [41]:
len(c_grouped[c_grouped["Shopping Mall"] > 0])

9

<h4>Create a new DataFrame for Shopping Mall data only</h4>

In [42]:
c_mall = c_grouped[["Neighborhoods","Shopping Mall"]]

In [43]:
c_mall.head()

,Neighborhoods,Shopping Mall
0,Alandur,0.000000
1,Anna Nagar,0.020000
2,"Ashok Nagar, Chennai",0.014706
3,Assisi Nagar,0.000000
4,Ayanavaram,0.000000


<h3>Cluster Neighborhoods</h3>

Run k-means to cluster the neighborhoods in Chennai into 3 clusters.

In [44]:
# set number of clusters
cclusters = 3

c_clustering = c_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=cclusters, random_state=0).fit(c_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 2, 2, 0, 0, 2, 0, 0, 0, 0], dtype=int32)

In [45]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
c_merged = c_mall.copy()

# add clustering labels
c_merged["Cluster Labels"] = kmeans.labels_

In [46]:
c_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
c_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Alandur,0.000000,0
1,Anna Nagar,0.020000,2
2,"Ashok Nagar, Chennai",0.014706,2
3,Assisi Nagar,0.000000,0
4,Ayanavaram,0.000000,0


In [47]:
# merge chennai_grouped with chennai_data to add latitude/longitude for each neighborhood
c_merged = c_merged.join(c_df.set_index("Neighborhood"), on="Neighborhood")

print(c_merged.shape)
c_merged.head() # check the last columns!

(68, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Alandur,0.000000,0,13.00013,80.20060
1,Anna Nagar,0.020000,2,13.08359,80.21020
2,"Ashok Nagar, Chennai",0.014706,2,13.03539,80.21220
3,Assisi Nagar,0.000000,0,13.16457,80.23274
4,Ayanavaram,0.000000,0,13.09883,80.23238


In [48]:
# sort the results by Cluster Labels
print(c_merged.shape)
c_merged.sort_values(["Cluster Labels"], inplace=True)
c_merged

(68, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Alandur,0.000000,0,13.000130,80.200600
34,Nazarethpettai,0.000000,0,13.037100,80.057550
35,Oragadam,0.000000,0,13.137440,80.153830
36,Padappai,0.000000,0,12.876997,80.048508
37,Palavakkam,0.000000,0,12.961390,80.257430
38,Pallavaram,0.000000,0,12.974440,80.148520
39,Pallikaranai,0.000000,0,12.955670,80.220800
41,Panambakkam,0.000000,0,13.077610,80.155830
42,Pattabiram,0.000000,0,13.123330,80.059440
43,Peerkankaranai,0.000000,0,12.912240,80.098950


<h4>Finally, let's visualize the resulting clusters</h4>

In [50]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(cclusters)
ys = [i+x+(i*x)**2 for i in range(cclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(c_merged['Latitude'], c_merged['Longitude'], c_merged['Neighborhood'], c_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [51]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

<h3>Examine Clusters</h3>

<h4>Cluster 0</h4>

In [52]:
c_merged.loc[c_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Alandur,0.0,0,13.000130,80.200600
34,Nazarethpettai,0.0,0,13.037100,80.057550
35,Oragadam,0.0,0,13.137440,80.153830
36,Padappai,0.0,0,12.876997,80.048508
37,Palavakkam,0.0,0,12.961390,80.257430
38,Pallavaram,0.0,0,12.974440,80.148520
39,Pallikaranai,0.0,0,12.955670,80.220800
41,Panambakkam,0.0,0,13.077610,80.155830
42,Pattabiram,0.0,0,13.123330,80.059440
43,Peerkankaranai,0.0,0,12.912240,80.098950


<h4>Cluster 1</h4>

In [53]:
c_merged.loc[c_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
33,Navalur,0.062500,1,12.84584,80.22648
55,Semmencherry,0.041667,1,12.86557,80.22051
40,Pammal,0.040000,1,12.96814,80.13359


<h4>Cluster 2</h4>

In [54]:
c_merged.loc[c_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
2,"Ashok Nagar, Chennai",0.014706,2,13.035390,80.212200
1,Anna Nagar,0.020000,2,13.083590,80.210200
64,Vadapalani,0.015385,2,13.052260,80.211200
65,Virugambakkam,0.017544,2,13.055900,80.193490
5,Chennai city,0.010000,2,13.074622,80.242777
18,Kodambakkam,0.020000,2,13.051520,80.224210
